In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")

df_a: df de amazon <br>
df_d: df de disney<br>
df_n: df de netflix<br>
df_h: df de hulu

In [2]:
df_a= pd.read_csv('Datasets/amazon_prime_titles.csv')
df_d= pd.read_csv('Datasets/disney_plus_titles.csv')
df_n= pd.read_json('Datasets/netflix_titles.json')
df_h= pd.read_csv('Datasets/hulu_titles.csv')

In [3]:
#lista con los df para iterar en los distintos procesos
dfs = [df_a,df_d,df_n,df_h]

## Criterios

Los criterio de la limpieza y pre-porcesamiento de los datos lo hice en base a las consultas que quiero responder.

Las consultas a realizar son:

+ Máxima duración según tipo de film (película/serie), por plataforma y por año:
    El request debe ser: get_max_duration(año, plataforma, [min o season])

+ Cantidad de películas y series (separado) por plataforma
    El request debe ser: get_count_plataform(plataforma)  
  
+ Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo.
    El request debe ser: get_listedin('genero')  
    Como ejemplo de género pueden usar 'comedy', el cuál deberia devolverles un cunt de 2099 para la plataforma de amazon.

+ Actor que más se repite según plataforma y año.
  El request debe ser: get_actor(plataforma, año)

### Analizo los datos que tengo:<br>
    - si el tipo de dato coincide con el Dtype
    - exitencia de conflicto entre los df
    - cantidad de valores faltantes

Los df comparten las mismas columnas.

In [4]:
df_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7585 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [5]:
df_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [6]:
df_n.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 894.5+ KB


La columna 'cast' tiene Dtype float64, cuando deberia ser object.
Además que no tiene datos por lo que cambaire lso ceros a 'Sin datos'

In [7]:
df_h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [8]:
df_h['cast'] = df_h['cast'].astype('object')

### Conflicto en show_id

- Los show_id de las ditintas plataformas se repiten por los que para evitar conficiones le agrego la letres de si plataforma delante del número. Ademas le quito la 's' que tiene delante del id.

In [9]:
letras = ['a','d','n','h']
#itera sobre los df en dfs y agrega la letra corespondiente a la plataforma usando el index de dfs.
for i,df in enumerate(dfs):
    df['show_id'] = df['show_id'].map(lambda x: letras[i]+x[1:])

### Doy formato a los datos que estan mal

- La columna 'duration' expresa la duracion de las peliculas en min o la cantidad de de temporadas si es una serie. <br>
Como es una varialbe cuantitativa discreta le cambio el Dtye a int64, para ello tengo que eliminar el 'min' y 'Season' o 'Seasons'

In [10]:
#funcion que deja solo la parte numerica de la columna 'duration'
def limpiar_datos_num(df):
    #df: dataframe que se va a limpiar
    
    #esta funcion elimina 'min' o 'Seasons' segun si es duracion en min o temporadas.
    def limp_segun_duracion(duracion):
        #verifico los que sean un valor nulo y los cambio por cero.
        if duracion == None or duracion == 'nan':
            return '0'
        #en caso de que la varible se float la vulevo str y la vuelvo a evaluar
        elif type(duracion) == float:
             return limp_segun_duracion(str(duracion))
        elif 'min' in duracion:
            return duracion[:-3]
        else:
            return duracion[:-7]
    #aplica limp_segun_duracion en la columna 'duration' mediante mapeo
    df['duration'] = df['duration'].map(lambda x:limp_segun_duracion(x))

In [11]:
limpiar_datos_num(df_a)
limpiar_datos_num(df_d)
limpiar_datos_num(df_n)
limpiar_datos_num(df_h)

Cambio el Dtype de la columna 'duration' y 'release_year' a uint16 por ser valores positivos menores al maximo permitido de ese tipo (65535). 

In [12]:
def dar_formato(df):
    columns_int =['release_year','duration']
    
    for col in columns_int:
        df[col] = df[col].astype('uint16')

In [13]:
dar_formato(df_a)
dar_formato(df_d)
dar_formato(df_n)
dar_formato(df_h)

# Normalizacion

In [14]:
def conseguir_val_unicos(df,col):
    '''
    df: dataframe 
    col: str del nimbre de la columna del df 

    Devulve los valores unicos de la columna en formato Serie de pandas.
    '''

    valores=[]
    lista_valores = []
    valores_df = df[col].values
    
    #llena lista_valores con listas del reparto de cada pelicula
    for val in valores_df:
        if type(val) == str:
            
            lista_valores.append(str.split(val,','))

    #llena valores con los actores
    for lista in lista_valores:
        for val in lista:
            valores.append(val.lstrip())#utlizo ltrip porque hay valores con un espacio delante
    
    #convierte la lista en una serie de pandas y elimina los repetidos
    valores = pd.Series(valores)
    valores = valores.drop_duplicates()
    return valores


In [24]:
def valores_col(lista_df,col):
    '''
    lista_df: lista de dataframes
    col: str

    Devuelve los valores unicos de todos los dataframes en lista_df de la columna col
    '''
    #serie vacia
    valores_col=pd.Series()

    #llena valores_col 
    for df in lista_df:
        valores_col=pd.concat([valores_col, conseguir_val_unicos(df,col)])
    
    # elimino los valores repetidos
    valores_col.drop_duplicates(inplace=True)
    #ordeno la lista
    valores_col.sort_values(inplace=True)
    return valores_col

 ### Normalizando la columna 'listed_in'

Enlisto las distintas categorias de los dataframes

In [25]:
categoria =  valores_col([df_a,df_d,df_n,df_h],'listed_in')

In [26]:
categoria.values

array(['Action', 'Action & Adventure', 'Action-Adventure',
       'Adult Animation', 'Adventure', 'Animals & Nature', 'Animation',
       'Anime', 'Anime Features', 'Anime Series', 'Anthology', 'Arthouse',
       'Arts', 'Biographical', 'Black Stories', 'British TV Shows',
       'Buddy', 'Cartoons', 'Children & Family Movies',
       'Classic & Cult TV', 'Classic Movies', 'Classics', 'Comedies',
       'Comedy', 'Coming of Age', 'Concert Film', 'Cooking & Food',
       'Crime', 'Crime TV Shows', 'Cult Movies', 'Dance', 'Disaster',
       'Documentaries', 'Documentary', 'Docuseries', 'Drama', 'Dramas',
       'Entertainment', 'Faith & Spirituality', 'Faith and Spirituality',
       'Family', 'Fantasy', 'Fitness', 'Game Show / Competition',
       'Game Shows', 'Health & Wellness', 'Historical', 'History',
       'Horror', 'Horror Movies', 'Independent Movies', 'International',
       'International Movies', 'International TV Shows', 'Kids',
       "Kids' TV", 'Korean TV Shows', 'LGBTQ'

Esta seria la normalizacion para categorias

In [28]:
valores_normalizar ={'Action & Adventure':['Action-Adventure','TV Action & Adventure'],
                    'Anime':['Animation','Anime', 'Anime Features', 'Anime Series'],
                    'Classic':['Classic & Cult TV', 'Classic Movies', 'Classics'],
                    'Comedy':['TV Comedies','Comedies'], 'Crime':['Crime TV Shows'], 'Documentary':['Documentaries'],
                    'Drama':['TV Dramas', 'Dramas'],'Faith & Spirituality':['Faith and Spirituality'],'Game Shows':['Game Show / Competition'],
                    'History':['Historical'], 'Horror': ['TV Horror','Horror Movies'],'International':['International Movies', 'International TV Shows'],
                    'Kids':["Kids' TV"],'LGBTQ':['LGBTQ Movies', 'LGBTQ+'],'Lifestyle':['Lifestyle & Culture'],'Mystery':['TV Mysteries'],
                    'Music':['Music & Musicals', 'Music Videos and Concerts', 'Musical'],'Reality':[ 'Reality TV'],
                    'Romance':['Romantic Movies','Romantic TV Shows'], 'Sports':['Sports Movies'],'Sci-Fi & Fantasy':['TV Sci-Fi & Fantasy'],
                    'Stand Up':['Stand-Up Comedy','Stand-Up Comedy & Talk Shows'],'Thriller':['TV Thrillers','Thrillers']
                   }

In [29]:
def normalizar_col(df,dict_valores,col):
    '''
    df: dataframe
    dict_valores: dict con su estructura = {valor_normalizado:valores_a_normalizar}
    col: str(nombre de la columna)

    Normaliza la columna col de df con los valores dados en dict_valores
    '''

    #itera sobre los valores de dict_valores
    for key,valores in dict_valores.items():
        #itera sobre los elementos en la columna 
        for str_valores in df[col]:
            #itera sobre los valores de cada elemnto del dicionario
            for val in valores:
                #verifica si el valor esta en el elemento
                if val in str_valores:
                    #remplaza el valor normalizado
                    df[col][df[col] == str_valores] = df[col][df[col] == str_valores].replace({str_valores:str_valores.replace(val,key)})



In [30]:
for df in dfs:
    normalizar_col(df,valores_normalizar,'listed_in')

In [31]:
categoria_normalizadas =  valores_col([df_a,df_d,df_n,df_h],'listed_in')

In [32]:
categoria_normalizadas.values

array(['Action', 'Action & Adventure', 'Adult Anime', 'Adventure',
       'Animals & Nature', 'Anime', 'Anthology', 'Arthouse', 'Arts',
       'Biographical', 'Black Stories', 'British TV Shows', 'Buddy',
       'Cartoons', 'Children & Family Movies', 'Classic', 'Comedy',
       'Coming of Age', 'Concert Film', 'Cooking & Food', 'Crime',
       'Cult Movies', 'Dance', 'Disaster', 'Documentary', 'Docuseries',
       'Drama', 'Entertainment', 'Faith & Spirituality', 'Family',
       'Fantasy', 'Fitness', 'Game Shows', 'Health & Wellness', 'History',
       'Horror', 'Independent Movies', 'International', 'Kids',
       'Korean TV Shows', 'LGBTQ', 'Late Night', 'Latino', 'Lifestyle',
       'Medical', 'Military and War', 'Movies', 'Music', 'Mystery',
       'News', 'Parody', 'Police/Cop', 'Reality', 'Romance',
       'Romantic Comedy', 'Sci-Fi & Fantasy', 'Science & Nature TV',
       'Science & Technology', 'Science Fiction', 'Series', 'Sitcom',
       'Sketch Comedy', 'Soap Opera / Melo

Cantidad de categorias se redujo de 120 a 84

# Lleno los datos faltantes con datos de las distintas plataformas

Los valores de inteser segun los criterio ya mencionados son:<br>
- duration(netflix:3,hulu:479)
- cast(amazon:1233,disney:190,netflix:825,hulu:3037)
- realise_year(completo)
- list_in(completo)

In [33]:
def completar_col(df,columna,df_comprobar):
    '''
    df: dataframe que queremos completar
    columna: str del la columna a revisar
    df_comprobar: dataframe en que se va a comprobar si hay datos

    Busca las peliculas que pertenecen a ambos dataframes, e itera sobre ellos para 
    completar los datos faltantes si es que en el valor de la columna dada es nulo en df
    y no es nulo en df_comprobar
    '''

    #mascara para filtrar los titulos con nan en la columna
    mask_nan = df[columna].isna() == True
    # genero una serie con las pelicula que tengan nan o none ne la columna dada por parametro
    films_nan = df['title'][mask_nan]
    #genero una serie de los titulos presentes en ambos df 
    mask_comp = films_nan.isin(df_comprobar['title'])
    film_comp = films_nan[mask_comp]
    
    #itero sobre los titulos que comparten ambos df
    for title in film_comp:
        valor_col_df = df[columna][df['title'] == title]
        if valor_col_df.isnull().values:  
            valor_col = df_comprobar[columna][df_comprobar['title'] == title]
            if  not valor_col.isnull().values:
                index = df[df['title'] == title].index
                index_col= list(df_h[0:0]).index(columna)
                df.iloc[index,index_col] = valor_col

In [34]:
def completar_df(df,lista_col,lista_df_comprobar):
    '''
    df: dataframe a completar
    lista_col: lista de las columnas a completar
    df_comprobar: dataframe con el que se quiere comprobar los datos faltantes

    Usa la funcion complera_col par lleanr a df en las columnas dadas en lista_col
    con los datos de los dataframes de lista_df_comprobar.
    '''

    for col in lista_col:
        for df_com in lista_df_comprobar:
            completar_col(df, col, df_com)

Netflix y Hulu tienen valores faltantes en 'duration' y 'cast' uso la funcion completar_df

In [25]:
completar_df(df_h,['duration','cast'],[df_a,df_d,df_n])

In [26]:
completar_df(df_n,['duration','cast'],[df_a,df_d,df_h])

A Dysney+ y Amazon solo falta completar la columna cast 

In [27]:
completar_df(df_a,['cast'],[df_n,df_d,df_h])
completar_df(df_d,['cast'],[df_n,df_a,df_h])

## Identificar a que plataforma pretenece

Para poder identificar a que plataforma pertence le agrego a las plataformas una columna con el nombre de la platforma

In [29]:
platformas = ['amazon','disney','netflix','hulu']
for i,df in enumerate(dfs):
    df['platform'] = np.array(platformas[i])

## Concateno

In [30]:
df_queries = pd.concat([df_a,df_d,df_n,df_h])

## Columnas sobrantres

Elimino las columnas 'director','country','date_added','rating' y 'description' que no aporta a las query requeridas.

In [32]:
df_queries.drop(['director', 'country', 'date_added','rating','description'],axis=1,inplace=True)

## Lleno los valores faltantes

In [33]:
df_queries.fillna('sin datos',inplace=True)

## Pongo en minuscula los valores

In [34]:
columnas =['type', 'title', 'cast', 'listed_in']
for col in columnas:
    df_queries[col] = df_queries[col].apply(lambda x : x.lower())

## Elimino los espacios al principio y final de los valores

In [35]:
columnas =['type', 'title', 'cast', 'listed_in']
for col in columnas:
    df_queries[col] = df_queries[col].apply(lambda x : x.rstrip().lstrip())

### ¿Conviene eliminar las peliculas o series duplicadas?

In [36]:
print('La cantidad de peliculas y seires repetidas son:',df_queries.shape[0]- df_queries['title'].unique().shape[0])
print('El porcentaje de peliculas repetidas es :',((df_queries.shape[0] - df_queries['title'].unique().shape[0]) / df_queries.shape[0]))

La cantidad de peliculas y seires repetidas son: 957
El porcentaje de peliculas repetidas es : 0.041612314114270806


Las peliculas o series repetidas no son muy pocas en comparacion al total.

## Exporto el df_query

Exporto el df_query y lo guardo en la carpeta 'Datasets' para ser utlizado por la api

In [38]:
df_queries.to_csv('app/df_procesado.csv',index=False)